In [1]:
import cv2
import numpy as np
import face_recognition
import os
import matplotlib.pyplot as plt
from simple_facerec import SimpleFacerec

In [2]:
path = "images"
images = []
classnames = []
myList = os.listdir(path)
for i in myList:
    curimg = cv2.imread(f'{path}/{i}')
    images.append(curimg)
    classnames.append(os.path.splitext(i)[0])
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [3]:
def findEncodings(images):
    encodelist = []
    for img in images:
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encodelist.append(encode)
    return encodelist

In [4]:
encodelistknown = findEncodings(images)
print(f"{len(encodelistknown)} Images encoded")

5 Images encoded


In [5]:
import cv2
from tensorflow.keras.utils import img_to_array
face_haar_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [6]:
#web cam
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    frames = cv2.resize(frame,(0,0),None,0.25,0.25)
    frames = cv2.cvtColor(frames,cv2.COLOR_BGR2RGB)
    height, width , channel = frame.shape
    gray_image= cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_haar_cascade.detectMultiScale(gray_image)
    faceloc = face_recognition.face_locations(frames)
    encode = face_recognition.face_encodings(frames,faceloc)
    try:
        for i,j in zip(encode,faceloc):
            matches = face_recognition.compare_faces(encodelistknown,i)
            facedis = face_recognition.face_distance(encodelistknown,i)
            matchindex = np.argmin(facedis)
            if matches[matchindex]:
                name = classnames[matchindex].upper()
                y1,x2,y2,x1 = j
                y1,x2,y2,x1 = (y1)*4,(x2)*4,(y2)*4,(x1)*4
                roi_gray = gray_image[y1-5:y2+5,x1-5:x2+5]
                roi_gray=cv2.resize(roi_gray,(48,48))
                image_pixels = img_to_array(roi_gray)
                image_pixels = np.expand_dims(image_pixels, axis = 0)
                image_pixels /= 255
                frame = cv2.rectangle(frame,(x1,y1),(x2,y2),(0,255,0),2)
                frame = cv2.rectangle(frame,(x1,y2+35),(x2,y2),(0,255,0),cv2.FILLED)
                frame = cv2.putText(frame,name,(x1,y2+35),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),2)
    except:
        pass
    cv2.imshow("WEBCAM", frame)
    key = cv2.waitKey(1)
    if key ==27:
        break
cap.release()
cv2.destroyAllWindows()